# Part 3 - Loading Data

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey
from sqlalchemy.orm import declarative_base, relationship


product_df = pd.read_csv("../data/warehouse_products.csv")
operations_df = pd.read_csv("../data/warehouse_daily.csv")

## Wide to Long

In [2]:
def select_one_strategy(df, underscore_strategy, shared_columns, strategy):
    
    column_list = [column for column in df.columns if column.endswith(underscore_strategy)]
    renamed_columns = {column: column.replace(underscore_strategy, "") for column in column_list}
    single_strat_df = df[shared_columns + column_list].rename(columns=renamed_columns)
    single_strat_df["strategy"] = strategy

    return single_strat_df

In [3]:
shared_columns_pro = ["date", "product_id", "warehouse_id", "demand"]

In [4]:
weekly_pro_df = select_one_strategy(product_df, "_weekly", shared_columns_pro, "weekly")
jit_pro_df = select_one_strategy(product_df, "_jit", shared_columns_pro, "JIT")
combined_product_df = pd.concat([weekly_pro_df, jit_pro_df], ignore_index=True)

In [5]:
shared_columns_op = ["date", "warehouse_id"]

In [9]:
weekly_op_df = select_one_strategy(operations_df, "_weekly", shared_columns_op, "weekly")
jit_op_df = select_one_strategy(operations_df, "_jit", shared_columns_op, "JIT")
combined_operations_df = pd.concat([weekly_op_df, jit_op_df], ignore_index=True)

## Schema

<center><img src="../schema.svg"></center>
<center><i>Schema created with <a href="https://dbdiagram.io/">dbdiagram.io</a></i></center>